*i Started out with nothin and i Still got most of it Left.*

–Seasick Steve

# Simulación
## Modelado de una inversión en CETES

In [ ]:
from datetime import date, timedelta
import requests

In [ ]:
# Reemplaza 'TU_TOKEN_AQUI' con tu token real de Banxico
# Lo puedes obtener en https://www.banxico.org.mx/SieAPIRest/service/v1/token
BANXICO_API_TOKEN = "PON AQUÍ TU TOKEN"

# Identificadores para diferentes series de datos
# SF43936: CETES a 28 días
# SF43939: CETES a 91 días
# SF43942: CETES a 182 días
# SP1: INPC
# Vamos a usar la de CETES a 28 días y la del INPC (inflación)
cetes_id = "SF43936"
inpc_id = "SP1"
series = f"{cetes_id},{inpc_id}"

# URL de la API de Banxico para consultar series de tiempo
url = f"https://www.banxico.org.mx/SieAPIRest/service/v1/series/{series}/datos"
headers = {
    'Bmx-Token': BANXICO_API_TOKEN,
    'Accept': 'application/json'
}

In [ ]:
response = requests.get(url, headers=headers)
response.json()

In [ ]:
series = response.json()["bmx"]["series"]
series

In [ ]:
len(series)

In [ ]:
type(series[0])

In [ ]:
for serie in series:
    datos = []
    for dato in serie["datos"]:
        fecha = dato["fecha"]
        valor = dato["dato"]
        try:
            fecha = date.strptime(fecha, "%d/%m/%Y")
            valor = float(valor)
        except ValueError:
            continue
        datos.append((fecha, valor))
    if serie["idSerie"] == inpc_id:
        inpc = datos
    elif serie["idSerie"] == cetes_id:
        cetes = datos      

In [ ]:
len(inpc), len(cetes)

In [ ]:
cetes == sorted(cetes), inpc == sorted(inpc)

In [ ]:
def buscar(datos, fecha, tope_sup=True):
    """
    Buscar, para una fecha dada, el valor correspondiente, en una lista de tuplas.
    El primer elemento de la tupla es la fecha, el segundo el valor.
    Las fechas se encuentran ordenadas de manera ascendente.
    Si no se encuentra la fecha exacta, se regresa el valor correspondiente a la fecha
    inmediatamente anterior (mayor fecha que sea menor o igual a la buscada).
    Si la fecha no se encuentra dentro del intervalo cubierto por la lista, regresa None,
    a menos que tope_sup sea False, en cuyo caso, regresa el último valor de la lista
    (correspondiente a la fecha más reciente).
    """
    for (f1, v1), (f2, v2) in zip(datos[:-1], datos[1:]):
        if f1 <= fecha < f2:
            valor = v1
            break
        if f1 > fecha:
            # Ya nos pasamos
            valor = None
            break
    else:
        if tope_sup:
            valor = None
        else:
            valor = v2
    return valor

In [ ]:
def valorar_portafolio(portafolio, fecha):
    """
    Devuelve el valor actual de un portafolio (a la fecha indicada).
    Un portafolio es una lista de inversiones, donde cada inversión se
    representa como se describe más adelante.
    """
    valor_total = 0
    for inversion in portafolio:
        valor_total += valorar_inversion(inversion, fecha)
    return valor_total

def valorar_inversion(inversion, fecha):
    """
    Devuelve el valor actualizado de una inversión a una fecha dada.
    La inversión se representa mediante un diccionario con claves: monto, tasa, vencimiento y plazo.
    """
    valor_inicial = inversion["monto"]
    tasa_diaria = inversion["tasa"] / 100 / 360
    if fecha >= inversion["vencimiento"]:
        tasa_efectiva = tasa_diaria * inversion["plazo"]
    else:
        dias_transcurridos = inversion["plazo"] - (inversion["vencimiento"] - fecha).days
        tasa_efectiva = tasa_diaria * dias_transcurridos
    valor_final = valor_inicial * (1 + tasa_efectiva)
    return valor_final

In [ ]:
"""
Estrategia de inversión:

- Meter a CETES el equivalente a $100 actuales cada martes durante 40 años.
- Comenzar en el primer martes de hace 40 años.
- Si no existe dato de CETES para esa fecha, brincársela.

Nota: Los CETES se subastan en martes, pero en la tabla vienen reportados al jueves (¡?)
"""

inversion_semanal = 100
# 01/ene de hace 40 años
fecha_inicial = date(date.today().year - 40, 1, 1)
fecha_final = date(date.today().year, date.today().month, 1) - timedelta(days=1)  # Último día del mes pasado
# Moverlo al martes
dia_semana = fecha_inicial.weekday() # 0: lunes, 1: martes, ...
dias = (3 - dia_semana + 7) % 7
fecha_inicial = fecha_inicial + timedelta(days=dias)
fecha_inicial, fecha_inicial.weekday()

In [ ]:
fecha = fecha_inicial
inpc_actual = buscar(inpc, date.today(), tope_sup=False)  # El más reciente, aunque no corresponda al mes
portafolio = []
historico = []
invertido = 0
disponible = 0
while fecha <= fecha_final:
    # Ajustar el depósito semanal por inflación
    inpc_historico = buscar(inpc, fecha, tope_sup=False)
    deposito = inversion_semanal / inpc_actual * inpc_historico
    invertido += deposito
    # Tenemos disponible ese depósito semanal
    disponible += deposito
    # Más cualquier inversión que haya madurado a la fecha
    for inversion in portafolio:
        if inversion["vencimiento"] <= fecha:
            disponible += valorar_inversion(inversion, fecha)
            inversion["monto"] = 0
    # "Limpiar" el portafolio
    portafolio = [inversion for inversion in portafolio if inversion["monto"] > 0]
    # Comprar CETES
    if (tasa := buscar(cetes, fecha)):
        portafolio.append({
            "monto": disponible,
            "tasa": tasa,
            "plazo": 28,
            "inicio": fecha,
            "vencimiento": fecha + timedelta(days=28),
        })
        disponible = 0
    # Llevar un registro del comportamiento de la inversión
    historico.append({
        "fecha": fecha,
        "invertido": invertido,
        "valor": valorar_portafolio(portafolio, fecha)
    })
    fecha += timedelta(days=7)
# Composición final del portafolio
portafolio

In [ ]:
# Valor actual del portafolio
f"{valorar_portafolio(portafolio, date.today()):,.2f}"


In [ ]:
import matplotlib.pyplot as plt

fechas = [dato["fecha"] for dato in historico]
invertido = [dato["invertido"] for dato in historico]
valor = [dato["valor"] for dato in historico]

plt.plot(fechas, invertido, label="Total invertido")
plt.plot(fechas, valor, label="Valor del portafolio")
plt.legend()
plt.show()

In [ ]:
historico[0]